# Docker Compose

## Features

# Production

## Gunicorn

### Creating the WSGI Entry Point

Moving along, for production environments, let's add Gunicorn, a production-grade WSGI server, to the requirements file:

    gunicorn==20.0.4

Next, let’s create a file that will serve as the entry point for our application. This will tell our Gunicorn server how to interact with the application.

Let’s call the file `wsgi.py`:

In [ ]:
from app import app

if __name__ == "__main__":
    app.run()

### Production Dockerfile

To use this file, create a new Dockerfile called Dockerfile.prod for use with production builds:

```Dockerfile
###########
# BUILDER #
###########

# pull official base image
FROM python:3.8.1-slim-buster as builder

# set work directory
WORKDIR /usr/src/app

# set environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1

# install system dependencies
RUN apt-get update && \
    apt-get install -y --no-install-recommends gcc

# install python dependencies
COPY ./requirements.txt .
RUN pip wheel --no-cache-dir --no-deps --wheel-dir /usr/src/app/wheels -r requirements.txt


#########
# FINAL #
#########

# pull official base image
FROM python:3.8.1-slim-buster

# create directory for the app user
RUN mkdir -p /home/app

# create the appropriate directories
ENV HOME=/home/app
ENV APP_HOME=/home/app/web
RUN mkdir $APP_HOME
WORKDIR $APP_HOME

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
COPY --from=builder /usr/src/app/wheels /wheels
COPY --from=builder /usr/src/app/requirements.txt .
RUN pip install --upgrade pip
RUN pip install --no-cache /wheels/*

# copy project
COPY . $APP_HOME

# run entrypoint.prod.sh
ENTRYPOINT ["python", "/home/app/web/wsgi.py"]
```

Here, we used a Docker multi-stage build to reduce the final image size. Essentially, builder is a temporary image that's used for building the Python wheels. The wheels are then copied over to the final production image and the builder image is discarded.

    docker build -t leon11sj/counter -f ./Dockerfile.prod .

### Preizkusimo delovanje

Since we still want to use Flask's built-in server in development, create a new compose file called `docker-compose.prod.yml` for production:

```yml
version: '3.7'
services:
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    ports:
      - "80:5000"
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    networks:  
      - backend
    

networks:
  frontend:
  backend:
```

    docker-compose -f docker-compose.prod.yml up

## Nginx as a reverse proxy

Next, let's add Nginx into the mix to act as a reverse proxy for Gunicorn to handle client requests as well as serve up static files.

Add the service to docker-compose.prod.yml:

      nginx:
        build: 
          context: ./nginx
          dockerfile: Dockerfile
        ports:
          - 80:8080
        depends_on:
          - web
        networks:
          - frontend

Then,  create the following files and folders:

    └── nginx
        ├── Dockerfile
        └── nginx.conf

`Dockerfile:`

    FROM nginx:1.17-alpine

    RUN rm /etc/nginx/conf.d/default.conf
    COPY nginx.conf /etc/nginx/conf.d

`nginx.conf:`

    upstream flask {
        server web:5000;
    }

    server {

        listen 8080;

        location / {
            proxy_pass http://flask;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header Host $host;
            proxy_redirect off;
        }

    }

https://www.patricksoftwareblog.com/how-to-configure-nginx-for-a-flask-web-application/

Then, update the web service, in docker-compose.prod.yml, replacing ports with expose:

```yml
version: '3.7'
services:
  nginx:
    build: 
      context: ./nginx
      dockerfile: Dockerfile
    ports:
      - 80:8080
    depends_on:
      - web
    networks:
      - frontend
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    expose:
      - 5000
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    networks:  
      - backend
    

networks:
  frontend:
  backend:

```

    docker-compose -f docker-compose.prod.yml up --build

## Persistent Redis data

https://docs.docker.com/compose/compose-file/#volume-configuration-reference

Prilagodimo compose file:

```yml
version: '3.7'
services:
  nginx:
    build: 
      context: ./nginx
      dockerfile: Dockerfile
    ports:
      - 80:8080
    depends_on:
      - web
    networks:
      - frontend
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    expose:
      - 5000
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    volumes:
      - redis-data:/data
    networks:  
      - backend
    

networks:
  frontend:
  backend:
  
volumes:
  redis-data:
 ```

Zaženemo in pokažemo, da se podatki ohranijo:

    docker-compose -f docker-compose.prod.yml up
    docker-compose -f docker-compose.prod.yml down
    docker-compose -f docker-compose.prod.yml up
    docker-compose -f docker-compose.prod.yml down --volumes
    docker-compose -f docker-compose.prod.yml up